In [2]:
import os
import fnmatch
import datetime
from datetime import datetime
import json
import pandas as pd
from joblib import dump, load
from sklearn.neighbors import KNeighborsClassifier
from finta import TA
#from sklearn.ensemble import RandomForestClassifier
#from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import metrics
import time
import requests
alpha_vantage_key = None
fmp_key_1 = None
fmp_key = None

In [9]:
small_cap_url = f'https://financialmodelingprep.com/api/v3/stock-screener?marketCapMoreThan=300000000&marketCapLowerThan=2000000001&limit=3&apikey={fmp_key}'
mid_cap_url = f'https://financialmodelingprep.com/api/v3/stock-screener?marketCapMoreThan=2000000000&marketCapLowerThan=10000000001&limit=3&apikey={fmp_key}'
large_cap_url = f'https://financialmodelingprep.com/api/v3/stock-screener?marketCapMoreThan=10000000000&limit=3&apikey={fmp_key}'
# find api that can screen by those

# https://api-v2.intrinio.com/securities/screen
# need to have top 3 of each sector(aka GICS group)/market cap pair
training_stocks = {
    'Consumer Cyclical':[]
    # 'Energy':[],
    # 'Technology':[],
    # 'Industrials':[],
    # 'Financial Services':[],
    # 'Basic Materials':[],
    # 'Communication Services':[],
    # 'Consumer Defensive':[],
    # 'Healthcare':[],
    # 'Real Estate':[],
    # 'Utilities':[],
    # 'Industrial Goods':[],
    # 'Financial':[],
    # 'Services':[],
    # 'Conglomerates':[]
}
for sector in training_stocks:
    url = small_cap_url + f'&sector={sector}'
    r = requests.get(url)
    data = r.json()
    for stock in data:
        training_stocks[sector].append(stock['symbol'])
    url = mid_cap_url + f'&sector={sector}'
    r = requests.get(url)
    data = r.json()
    for stock in data:
        training_stocks[sector].append(stock['symbol'])
    url = large_cap_url + f'&sector={sector}'
    r = requests.get(url)
    data = r.json()
    for stock in data:
        training_stocks[sector].append(stock['symbol'])
    time.sleep(0)

In [10]:
print(training_stocks)

{'Consumer Cyclical': ['SBH', 'BKE', 'DFH', 'GIL.TO', 'LAD', 'LEA', 'AMZN', 'TSLA', 'BABA']}


In [53]:
def get_and_clean_data(stock):
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&datatype=csv&adjusted=true&symbol={stock}&outputsize=full&apikey={alpha_vantage_key}'
    data = pd.read_csv(url, index_col='timestamp')
    data_price = data.copy()
    data_rsi = TA.STOCHRSI(data)
    data_macd = TA.MACD(data)
    data_stoch = TA.STOCH(data)
    data_vzo = TA.VZO(data)
    url = f'https://www.alphavantage.co/query?function=CCI&time_period=60&datatype=csv&symbol={stock}&interval=daily&apikey={alpha_vantage_key}'
    data_cci = pd.read_csv(url, index_col='time')
    url = f'https://www.alphavantage.co/query?function=CONSUMER_SENTIMENT&datatype=csv&apikey={alpha_vantage_key}'
    data_sent = pd.read_csv(url, index_col='timestamp')
    # url = f'https://www.alphavantage.co/query?function=UNEMPLOYMENT&datatype=csv&apikey={alpha_vantage_key}'
    # data_unemployment = pd.read_csv(url, index_col='timestamp')
    data = data.join(data_rsi, how='left') 
    data = data.join(data_macd, how='left') 
    data = data.join(data_stoch, how='left') 
    data = data.join(data_vzo, how='left') 
    data = data.join(data_cci, how='left') 
    data = data.join(data_sent, how='left')
    # data = data.join(data_unemployment, how='left')
    data = data.drop(labels=['open', 'high', 'low', 'close', 'volume'], axis=1)
    scaler = preprocessing.MinMaxScaler() 
    scaled_values = scaler.fit_transform(data) 
    data.loc[:,:] = scaled_values
    data = data.join(data_price, how='left')
    data.interpolate(axis='columns', inplace=True)
    data.dropna(inplace=True)
    return data

In [54]:
data = get_and_clean_data('SBH')

In [55]:
data.head()
data.count()

14 period stochastic RSI.    3710
MACD                         3710
SIGNAL                       3710
14 period STOCH %K           3710
VZO                          3710
CCI                          3710
value                        3710
open                         3710
high                         3710
low                          3710
close                        3710
volume                       3710
dtype: int64

In [57]:
def add_ratings_to_data(df: pd.DataFrame):
    shift = 30 #30 days, 6.5 hrs/day, 2 half-hrs/hr
    df['rating'] = 'hold'
    df.loc[((df['open'] < df.shift(shift)['open']) & ((df.shift(shift)['open'] - df['open']) / df['open'] * 100 > 10)), 'rating'] = 'mega buy'
    df.loc[((df['open'] < df.shift(shift)['open']) & ((df.shift(shift)['open'] - df['open']) / df['open'] * 100 > 2)), 'rating'] = 'buy'
    df.loc[((df['open'] > df.shift(shift)['open']) & ((df['open'] - df.shift(shift)['open']) / df['open'] * 100 > 10)), 'rating'] = 'mega sell'
    df.loc[((df['open'] > df.shift(shift)['open']) & ((df['open'] - df.shift(shift)['open']) / df['open'] * 100 > 2)), 'rating'] = 'sell'
    return df

In [61]:
data_2 = add_ratings_to_data(data)
data_2.rating.unique()
data.head()

,14 period stochastic RSI.,MACD,SIGNAL,14 period STOCH %K,VZO,CCI,value,open,high,low,close,volume,rating
timestamp,,,,,,,,,,,,,
2021-08-16,1.000000,0.549226,0.524885,0.793269,0.725182,0.462702,9.706351,18.95,19.145,18.76,18.96,717132.0,hold
2021-08-13,0.948464,0.568575,0.528461,0.811060,0.755651,0.472568,9.956284,19.44,19.480,18.96,19.07,461822.0,hold
2021-08-12,0.903210,0.593045,0.536799,0.882114,0.793345,0.489582,10.054791,19.62,19.770,19.13,19.48,620655.0,hold
2021-08-11,0.858244,0.611890,0.547626,0.890244,0.821998,0.477096,9.898548,19.32,19.515,19.08,19.50,521757.0,hold
2021-08-10,0.809897,0.623289,0.558764,0.849593,0.677070,0.451935,9.510967,18.57,19.460,18.50,19.40,750068.0,hold


In [65]:
def get_stock_category(stock):
    url = f'https://financialmodelingprep.com/api/v3/profile/{stock}?apikey={fmp_key}'
    data = requests.get(url).json()
    sector = data[0]['sector']
    cap = data[0]['mktCap']
    if cap > 10_000_000_000:
        cap = 'large'
    elif cap < 10_000_000_000 and cap > 2_000_000_000:
        cap = 'mid'
    else:
        cap = 'small'
    sector = sector.replace(' ', '')
    category = f'{sector}_{cap}'
    return category

In [66]:
get_stock_category('SBH')

'ConsumerCyclical_small'